# Libraries used:

In [ ]:
import cv2 as cv
import mediapipe as mp
import os
import csv

# Initialize Mediapipe Hands object // MediaPipe setup

In [ ]:
mp_hands = mp.solutions.hands 
mp_drawing = mp.solutions.drawing_utils 
# Initializes the hand detection model for a video, one hand at most, and a minimum detection confidence of 0.5
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)

# Create folder, create csv file, collect x,y,z coordinates, normalize the data relative to the wrist coordinates, and write the data to the csv file

In [ ]:
DATA_PATH = "asl_dataset"
os.makedirs(DATA_PATH, exist_ok=True) 

labels = [chr(i) for i in range(ord('A'), ord('Z') + 1)] 
num_samples = 300 # Number of samples per label

File_PATH = os.path.join(DATA_PATH, 'asl_data.csv')
with open(File_PATH, 'w', newline= '') as f:
    
    print("The file \"asl_data.csv\" has been created/opened successfully.")
    writer = csv.writer(f)

    header = ['label']

    for i in range(21): # These are the 21 landmarks of the hand solution
        header += [f'x{i}', f'y{i}', f'z{i}'] 

    writer.writerow(header)

    webCam = cv.VideoCapture(0)

    for label in labels:
        print(f"Collecting data for sign {label}")

        cv.waitKey(5000) # 5 seconds to get the sign ready
        collected_count = 0

        while(collected_count < num_samples):

            success, frame = webCam.read() #gets a frame and indicates if it was successful

            if not success:
                print("could not read frame from webcam. Exiting...")
                break
            
            frame = cv.flip(frame, 1) # flip around y axis for mirror image

            #convert the BGR image to RGB because mediapipe needs RGB images
            rgb_frame = cv.cvtColor(frame, cv.COLOR_BGR2RGB) 
            result = hands.process(rgb_frame) # Process the frame to detect hands

            if result.multi_hand_landmarks: # if hands are detected:

                hand_landmarks = result.multi_hand_landmarks[0] # get the first hand detected

                # Display the landmarks on the camera
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)


                # Normalizing the data
            
                wristCoords = hand_landmarks.landmark[0]
            
                landmarks_data = []
            
                for lm in hand_landmarks.landmark:
                    normalized_x = lm.x - wristCoords.x
                    normalized_y = lm.y - wristCoords.y
                    normalized_z = lm.z - wristCoords.z
                    landmarks_data.extend([normalized_x, normalized_y, normalized_z])

                # Write label and landmarks data to the CSV file
                writer.writerow([label] + landmarks_data)

                collected_count += 1
                print(f"Collected sample {collected_count} for sign {label}")

            cv.putText(frame, f"Collected {collected_count} / {num_samples} For sign {label}", (10, 30), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv.LINE_AA)
            cv.imshow('Data Collection', frame)
            if cv.waitKey(1) == ord('q'):
                webCam.release()
                cv.destroyAllWindows()
                cv.waitKey(500)
                exit()

    webCam.release()
    cv.destroyAllWindows()
    cv.waitKey(500)
